# 📊 模型评估与可视化分析模块 (3_eval.ipynb)

## 📋 模块功能
对推荐系统进行**全面的评估分析和可视化**，量化各组件的效果并生成直观的分析报告。

## 🎯 核心功能
1. **📈 性能指标可视化**: Recall@K、NDCG@K指标对比
2. **🔬 消融实验分析**: 量化各召回策略和排序模型的贡献
3. **🌟 特征重要性分析**: 识别关键特征和优化方向
4. **📊 多维度对比**: 不同策略组合的效果对比

## 🎨 可视化要求
- 使用 **matplotlib** 进行绘图
- 每个图表**独立绘制**，便于分析
- 采用**系统默认配色**，专业简洁
- 支持**高分辨率输出**，适合报告使用

## 🔧 输入数据
- `metrics_ablation.csv`: 消融实验指标数据
- `feature_importance.csv`: 特征重要性数据（可选）

## 📈 输出图表
- 消融实验柱状图
- 指标对比折线图  
- 特征重要性排序图
- 性能提升分析图

In [6]:

# =============================================================================
# 环境配置与依赖导入
# =============================================================================
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置图表样式
plt.style.use('default')  # 使用默认样式
sns.set_palette("husl")   # 设置调色板

# 配置参数
OUTDIR = '../x'
print(f'📁 数据目录: {OUTDIR}')
print(f'⏰ 分析时间: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

# 检查必要文件
required_files = ['metrics_ablation.csv']
optional_files = ['feature_importance.csv']

print("🔍 检查数据文件...")
for file in required_files:
    if not os.path.exists(f'{OUTDIR}/{file}'):
        raise FileNotFoundError(f"❌ 缺少必要文件: {file}，请先运行 2_rank.ipynb")
    print(f"  ✅ {file}")

for file in optional_files:
    if os.path.exists(f'{OUTDIR}/{file}'):
        print(f"  ✅ {file} (可选)")
    else:
        print(f"  ⚠️  {file} (可选，未找到)")

print("✅ 环境配置完成")
    

📁 数据目录: ../x
⏰ 分析时间: 2025-09-26 20:16:05
🔍 检查数据文件...
  ✅ metrics_ablation.csv
  ✅ feature_importance.csv (可选)
✅ 环境配置完成


In [7]:
# =============================================================================
# 数据加载与预处理
# =============================================================================
print("📖 正在加载评估数据...")

# 加载消融实验指标
metrics = pd.read_csv(f'{OUTDIR}/metrics_ablation.csv')

print("✅ 数据加载完成")
print(f"📊 消融实验指标数据:")
print(f"  数据形状: {metrics.shape}")
print(f"  包含列: {list(metrics.columns)}")

# 显示数据概览
print(f"\n📋 消融实验结果预览:")
display(metrics)

# 数据质量检查
print(f"\n🔍 数据质量检查:")
print(f"  缺失值: {metrics.isnull().sum().sum()}")
print(f"  重复行: {metrics.duplicated().sum()}")

# 尝试加载特征重要性数据
feature_importance = None
if os.path.exists(f'{OUTDIR}/feature_importance.csv'):
    try:
        feature_importance = pd.read_csv(f'{OUTDIR}/feature_importance.csv')
        print(f"  📊 特征重要性数据: {feature_importance.shape}")
    except Exception as e:
        print(f"  ⚠️  特征重要性数据加载失败: {e}")
else:
    print(f"  ℹ️  未找到特征重要性数据，将跳过相关分析")


📖 正在加载评估数据...
✅ 数据加载完成
📊 消融实验指标数据:
  数据形状: (3, 5)
  包含列: ['setting', 'hr50', 'mrr50', 'ndcg50', 'total_score']

📋 消融实验结果预览:


,setting,hr50,mrr50,ndcg50,total_score
0,A1_covisit_only_NoRank,0.137074,0.046309,0.066506,0.103798
1,A1_multi_NoRank,0.247800,0.090065,0.124671,0.189897
2,A2_multi_Ranker,0.278800,0.223163,0.235954,0.258464



🔍 数据质量检查:
  缺失值: 0
  重复行: 0
  📊 特征重要性数据: (9, 2)


In [8]:
# 图1：Recall@50 消融柱状图plt.figure()x = np.arange(len(metrics))vals = metrics['hr50'].valuesplt.bar(x, vals)plt.xticks(x, metrics['setting'], rotation=20, ha='right')plt.title('Ablation: Recall@50')plt.ylabel('Recall@50')plt.tight_layout()plt.show()    

KeyError: 'recall_at_50'

<Figure size 640x480 with 0 Axes>

In [ ]:
# 图2：NDCG@50 消融柱状图plt.figure()x = np.arange(len(metrics))vals = metrics['ndcg50'].valuesplt.bar(x, vals)plt.xticks(x, metrics['setting'], rotation=20, ha='right')plt.title('Ablation: NDCG@50')plt.ylabel('NDCG@50')plt.tight_layout()plt.show()    

In [ ]:

# （可选）图3：LightGBM 特征重要性 Top10
import os, pandas as pd, matplotlib.pyplot as plt, numpy as np

fi_path = f'{OUTDIR}/feature_importance.csv'
if os.path.exists(fi_path):
    fi = pd.read_csv(fi_path).sort_values('importance', ascending=False).head(10)
    plt.figure()
    plt.barh(fi['feature'][::-1], fi['importance'][::-1])
    plt.title('Feature Importance (Top 10)')
    plt.tight_layout()
    plt.show()
else:
    print('feature_importance.csv not found (LightGBM may not be installed).')
    